In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Chapter 1: Parse HTML with BeautifulSoup

Summary of this notebook here: 
    We'll study the page that we have, using BeautifulSoup to parse structured HTML, extract the 200 top grossing movie names and their individual page links
    Then we will save everything in a flatfile

```
You didn't write that awful page. You're just trying to get some data out of it. Beautiful Soup is here to help.
```

** Beautiful Soup :** 
* sits atop an HTML or XML parser, 
* providing Pythonic idioms for iterating, searching, and modifying the parse tree
* handles text encodings automatically (always utf-8 out)
Beautiful Soup [Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/):  

In [5]:
import requests

url = 'http://www.boxofficemojo.com/alltime/adjusted.htm'
response = requests.get(url)

In [6]:
response

<Response [200]>

In [10]:
# if needed: !conda install beautifulsoup4 -y

Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/lingqiangkong/miniconda2/envs/py3:

The following NEW packages will be INSTALLED:

    beautifulsoup4: 4.6.0-py36_0

beautifulsoup4 100% |################################| Time: 0:00:00   3.48 MB/s


In [12]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(response.text, 'html.parser')

In [22]:
soup.html.head

<head>
<title>All Time Box Office Adjusted for Ticket Price Inflation</title>
<meta content="box, office, all, time, inflation, adjusted, ticket, price, report, movie, film" name="keywords"/>
<meta content="All time box office adjusted for ticket price inflation." name="description"/>
<link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
<link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/></head>

In [38]:
soup('table')[3]

<table border="0" cellpadding="5" cellspacing="1" width="95%"><tr bgcolor="#dcdcdc"><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=rank&amp;order=ASC&amp;adjust_yr=2017&amp;p=.htm">Rank</a></font></td><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=title&amp;order=ASC&amp;adjust_yr=2017&amp;p=.htm">Title (click to view)</a></font></td><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=studio&amp;order=ASC&amp;adjust_yr=2017&amp;p=.htm">Studio</a></font></td><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=adjustedgross&amp;order=ASC&amp;adjust_yr=2017&amp;p=.htm"><b>Adjusted Gross</b></a></font></td><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=gross&amp;order=DESC&amp;adjust_yr=2017&amp;p=.htm">Unadjusted Gross</a></font></td><td align="center"><font size="2"><a href="/alltime/adjusted.htm?sort=year&amp;order=DESC&amp;adjust_yr=2017&amp;p=.htm">Year^</a></font></td></tr><tr bgc

In [42]:
m = soup('table')[3]('tr')[1]('td')[1]

In [43]:
m

<td><font size="2"><a href="/movies/?id=gonewiththewind.htm"><b>Gone with the Wind</b></a></font></td>

In [45]:
m.a['href']

'/movies/?id=gonewiththewind.htm'

In [57]:
m.b.string

'Gone with the Wind'